In [1]:
import pandas as pd
import os
import numpy as np
import gurobipy as grb
os.sys.path = ['..\\']+ os.sys.path
from InstanceReader import read_instance


In [68]:
root_path = r'..\..\instances\7-10'
instance_i = 10
instance_name = f"instance{instance_i}.xls"

In [69]:
instances = read_instance(os.path.join(root_path, instance_name))
instances

{'visitor_info':    visitor_index time_limit
 0              0        189
 1              1        178
 2              2        221
 3              3        174
 4              4        219
 5              5        223
 6              6        202
 7              7        180
 8              8        135
 9              9        214,
 'attraction_info':   attraction_index attraction_reward serve_time
 0                e                 0          0
 1                0                 2          9
 2                1                 7         29
 3                2                 1         47
 4                3                 8         37
 5                4                11         44
 6                h                 0          0,
 'attraction_distance_map':      e    0   1    2    3   4    h
 e    0  114  76  105  114  18    0
 0  114    0  96   37  134  96  114
 1   76   96   0   59   38  58   76
 2  105   37  59    0   97  87  105
 3  114  134  38   97    0  96  114
 4   18  

In [8]:
[x for x in instances['attraction_info']['attraction_index']]

['e', 0, 1, 2, 3, 4, 'h']

In [27]:
instances['visitor_info']['visitor_index'].values

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [70]:
attraction_resource_num = 2
inf = max(instances['visitor_info']['time_limit'])
class visitor:
    def __init__(self, visitor_i_info):
        self.visitor_i = visitor_i_info['visitor_index']
        self.time_limit = visitor_i_info['time_limit']
    
    def initialize_var(self, m: grb.Model, attraction_info, visitor_info):
        temp = self.enumberate_x(attraction_info)
        self.x_var_dict = m.addVars(temp['var_index'], vtype=grb.GRB.BINARY, name=temp['var_name'])
        
        temp = self.enumberate_s_ij(attraction_info)
        self.start_time_dict = m.addVars(temp['start_index'], vtype=grb.GRB.INTEGER, lb=0, name=temp['start_name'])
        
        temp = self.enumberate_z(attraction_info)
        self.z_dict = m.addVars(temp['z_index'], vtype=grb.GRB.BINARY, name=temp['z_name'])
        
        temp = self.enumberate_q(attraction_info, visitor_info)
        self.q_dict = m.addVars(temp['q_index'], vtype=grb.GRB.BINARY, name=temp['q_name'])
        
    
    def enumberate_x(self, attraction_info):
        visitor_x_index = [
            (j, k)
            for j in  attraction_info['attraction_index'][:-1]
            for k in attraction_info['attraction_index'][1:] if j !=k 
        ]
        
        visitor_x_name = [
            f"x-v:{self.visitor_i}_{j}->{k}"
            for j in  attraction_info['attraction_index'][:-1]
            for k in attraction_info['attraction_index'][1:] if j !=k 
        ]
        return {'var_index': visitor_x_index, 'var_name': visitor_x_name}
        
    def enumberate_s_ij(self, attraction_info):
        start_time_j_index = attraction_info['attraction_index'].values.tolist()
        start_time_j_name = [f"s-v:{self.visitor_i}_StartTime:{j}" for j in  start_time_j_index]
        return {'start_index': start_time_j_index, 'start_name': start_time_j_name}
    
    def enumberate_z(self, attraction_info):
        z_index = [
            (j, m)
            for j in  attraction_info['attraction_index'][1:-1]
            for m in range(attraction_resource_num)
        ]
        z_name = [
            f"z-v:{self.visitor_i}_A:{j}_m:{m}"
            for j in  attraction_info['attraction_index'][1:-1]
            for m in range(attraction_resource_num)
        ]
        return {'z_index': z_index, 'z_name': z_name}
    
    def enumberate_q(self, attraction_info, visitor_info):
        q_index = [
            (self.visitor_i, l, j, m)
            for l in visitor_info['visitor_index'].values if l != self.visitor_i
            for j in attraction_info['attraction_index'][1:-1]
            for m in range(attraction_resource_num)
        ]
        q_name = [
            f"q-v:{self.visitor_i}_{l}_{j}_{m}"
            for l in visitor_info['visitor_index'].values if l != self.visitor_i
            for j in attraction_info['attraction_index'][1:-1]
            for m in range(attraction_resource_num)
        ]
        
        return {'q_index': q_index, 'q_name': q_name}
        

class MAOPCC:
    def __init__(self, attraction_info, vistor_info, attraction_distance_map, model_name='MAOPCC'):
        self.attraction_info = attraction_info
        self.visitor_info = vistor_info
        self.attraction_distance_map = attraction_distance_map
        
        self.m = grb.Model(model_name)
        self.visitors_list = None
        
    def initialize_vistors(self):
        self.visitors_list = [
            visitor(self.visitor_info.loc[visitor_i])
            for visitor_i in range(len(self.visitor_info))
        ]
        
        for visitor_i in self.visitors_list:
            visitor_i.initialize_var(m=self.m, attraction_info=self.attraction_info, visitor_info=self.visitor_info)
            
    def set_objective(self):
        all_reaward = sum([
            sum([
                self.visitors_list[visitor_i].x_var_dict[xijk] * self.attraction_info['attraction_reward'][
                    self.attraction_info['attraction_index']==xijk[-1]].item()
                for xijk in self.visitors_list[visitor_i].x_var_dict
            ])
            for visitor_i in range(len(self.visitors_list))
        ])
        self.m.setObjective(all_reaward, grb.GRB.MAXIMIZE)
        
    
    def add_constraints(self):
        # constraint ②
        self.m.addConstrs(
        (
            visitor_i.start_time_dict[j] <= visitor_i.time_limit
            for j in self.attraction_info['attraction_index']
            for visitor_i in self.visitors_list), name='C[2]'
        )
        
        # constraint ③
        self.m.addConstrs(
            (
                sum([
                    visitor_i.x_var_dict[xi0k] 
                    for xi0k in visitor_i.x_var_dict if xi0k[0] == 'e' 
                ]) == 1
                for visitor_i in self.visitors_list
            ), name='C[3]'
        )
        
        # constraint ④
        self.m.addConstrs(
            (
                sum([
                    visitor_i.x_var_dict[xijH1]
                    for xijH1 in visitor_i.x_var_dict if xijH1[-1] == 'h'
                ]) == 1
                for visitor_i in self.visitors_list
            ),name='C[4]'
        )
        
        # constraint ⑤
        self.m.addConstrs(
            (
                sum([
                    visitor_i.x_var_dict[xjk]
                    for xjk in visitor_i.x_var_dict if xjk[-1] == k 
                ]) <=1
                for k in self.attraction_info['attraction_index'][1:-1]
                for visitor_i in self.visitors_list
            ), name='C[5]'
        )
        
        self.m.addConstrs(
            (
                sum([
                    visitor_i.x_var_dict[xkj]
                    for xkj in visitor_i.x_var_dict if xkj[0] == k 
                ]) <=1
                for k in self.attraction_info['attraction_index'][1:-1]
                for visitor_i in self.visitors_list
            ), name='C[5]'
        )
        
        self.m.addConstrs(
            (
                sum([
                    visitor_i.x_var_dict[xjk]
                    for xjk in visitor_i.x_var_dict if xjk[-1] == k
                ]) == sum([
                    visitor_i.x_var_dict[xkj]
                    for xkj in visitor_i.x_var_dict if xkj[0] == k
                ])
                for k in self.attraction_info['attraction_index'][1:-1]
                for visitor_i in self.visitors_list
            ), name='C[5]'
        )
        
        # constraint 6
        
#         self.m.addConstrs(
#             (
                
#                 for k in self.attraction_info['attraction_index'][1:]
#                 for j in self.attraction_info['attraction_index'][0:-1] if k != j
#                 for i in range(len(self.visitors_list))
#             ), name='C[6]'
#         )
        
        # constraint ⑦
        self.m.addConstrs(
            (
                sum([
                    visitor_i.x_var_dict[xjk]
                    for xjk in visitor_i.x_var_dict if xjk[0] == j
                ]) == sum([
                    visitor_i.z_dict[z_ijm]
                    for z_ijm in visitor_i.z_dict if z_ijm[0] == j
                ])
                for j in self.attraction_info['attraction_index'][1:-1]
                for visitor_i in self.visitors_list
            ), name='C[7]'
        )
        
        # constraint ⑧
        self.m.addConstrs(
            (
                visitor_i.start_time_dict[x_ikj[-1]] - visitor_i.start_time_dict[x_ikj[0]] >= inf * (visitor_i.x_var_dict[x_ikj] - 1) + self.attraction_distance_map.loc[x_ikj[0], x_ikj[-1]] + 
                self.attraction_info['serve_time'][self.attraction_info['attraction_index'] == x_ikj[0]].item()
                for visitor_i in self.visitors_list
                for x_ikj in visitor_i.x_var_dict 
            ), name='C[8]'
        )
        
        # constraint ⑨
        self.m.addConstrs(
            (
                self.visitors_list[i].q_dict[(i,l,j, m)] + self.visitors_list[l].q_dict[(l, i, j, m)] >=
                self.visitors_list[i].z_dict[(j, m)] + self.visitors_list[l].z_dict[(j, m)] - 1
                for i in range(len(self.visitors_list))
                for l in range(len(self.visitors_list)) if i != l
                for j in self.attraction_info['attraction_index'][1:-1]
                for m in range(attraction_resource_num)
            ), name='C[9]'
        )
        
        # constraint ⑩
        self.m.addConstrs(
            (
                self.visitors_list[i].q_dict[(i,l,j, m)] + self.visitors_list[l].q_dict[(l, i, j, m)] <=
                0.5 * (self.visitors_list[i].z_dict[(j, m)] + self.visitors_list[l].z_dict[(j, m)])
                for i in range(len(self.visitors_list))
                for l in range(len(self.visitors_list)) if i != l
                for j in self.attraction_info['attraction_index'][1:-1]
                for m in range(attraction_resource_num)
            ), name='C[10]'
        )
        
        # constraint 11
        self.m.addConstrs(
            (
                self.visitors_list[l].start_time_dict[j] - self.visitors_list[i].start_time_dict[j] >=
                self.attraction_info['serve_time'][self.attraction_info['attraction_index'] == j].item() - inf * 
                (1 - sum([
                    self.visitors_list[i].q_dict[q_iljm]
                    for q_iljm in self.visitors_list[i].q_dict if (q_iljm[1] == l and q_iljm[-2] == j)
                ])
                )
                for i in range(len(self.visitors_list))
                for l in range(len(self.visitors_list)) if i != l
                for j in self.attraction_info['attraction_index'][1:-1]
            ), name='C[11]'
        )
        
    def opitimize(self):
        self.m.optimize()
        
        
        
        
        
        
        

In [71]:
%%time
maopcc = MAOPCC(attraction_info=instances['attraction_info'], vistor_info=instances['visitor_info'], 
       attraction_distance_map=instances['attraction_distance_map'])
# maopcc.initialize_vistors()
# maopcc.set_objective()
# maopcc.add_constraints()

Wall time: 19.3 ms


In [72]:
%%time
maopcc.initialize_vistors()

Wall time: 24.2 ms


In [73]:
%%time
maopcc.set_objective()

Wall time: 52.1 ms


In [74]:
%%time
maopcc.add_constraints()

Wall time: 336 ms


In [75]:
%%time
maopcc.opitimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2850 rows, 1380 columns and 11470 nonzeros
Model fingerprint: 0x7476bdeb
Variable types: 0 continuous, 1380 integer (1310 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 2408 rows and 1087 columns
Presolve time: 0.05s
Presolved: 442 rows, 293 columns, 1576 nonzeros
Variable types: 0 continuous, 293 integer (268 binary)
Found heuristic solution: objective 36.0000000

Root relaxation: objective 1.100000e+02, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Ob

In [23]:
maopcc.m.getVars()[0].varName

'x-v:0_e->0'

# show the solution

In [76]:
solutions = {var.varName: var.x for var in  maopcc.m.getVars() if var.x>0}
solutions

{'x-v:0_e->4': 1.0,
 'x-v:0_4->h': 1.0,
 's-v:0_StartTime:0': 106.0,
 's-v:0_StartTime:2': 89.0,
 's-v:0_StartTime:3': 90.0,
 's-v:0_StartTime:4': 20.0,
 's-v:0_StartTime:h': 189.0,
 'z-v:0_A:4_m:1': 1.0,
 'q-v:0_1_4_1': 1.0,
 'q-v:0_3_4_1': 1.0,
 'q-v:0_9_4_1': 1.0,
 'x-v:1_e->4': 1.0,
 'x-v:1_4->h': 1.0,
 's-v:1_StartTime:0': 95.0,
 's-v:1_StartTime:1': 61.0,
 's-v:1_StartTime:2': 89.0,
 's-v:1_StartTime:3': 90.0,
 's-v:1_StartTime:4': 64.0,
 's-v:1_StartTime:h': 178.0,
 'z-v:1_A:4_m:1': 1.0,
 'q-v:1_3_4_1': 1.0,
 'q-v:1_9_4_1': 1.0,
 'x-v:2_e->1': 1.0,
 'x-v:2_1->h': 1.0,
 's-v:2_StartTime:0': 118.0,
 's-v:2_StartTime:1': 76.0,
 's-v:2_StartTime:2': 89.0,
 's-v:2_StartTime:3': 90.0,
 's-v:2_StartTime:4': 72.0,
 's-v:2_StartTime:h': 221.0,
 'z-v:2_A:1_m:1': 1.0,
 'x-v:3_e->4': 1.0,
 'x-v:3_4->h': 1.0,
 's-v:3_StartTime:0': 91.0,
 's-v:3_StartTime:1': 57.0,
 's-v:3_StartTime:2': 85.0,
 's-v:3_StartTime:3': 90.0,
 's-v:3_StartTime:4': 108.0,
 's-v:3_StartTime:h': 174.0,
 'z-v:3_A:4_m:1

In [78]:
instances

{'visitor_info':    visitor_index time_limit
 0              0        189
 1              1        178
 2              2        221
 3              3        174
 4              4        219
 5              5        223
 6              6        202
 7              7        180
 8              8        135
 9              9        214,
 'attraction_info':   attraction_index attraction_reward serve_time
 0                e                 0          0
 1                0                 2          9
 2                1                 7         29
 3                2                 1         47
 4                3                 8         37
 5                4                11         44
 6                h                 0          0,
 'attraction_distance_map':      e    0   1    2    3   4    h
 e    0  114  76  105  114  18    0
 0  114    0  96   37  134  96  114
 1   76   96   0   59   38  58   76
 2  105   37  59    0   97  87  105
 3  114  134  38   97    0  96  114
 4   18  

In [40]:
solutions[ 'x-v:0_e->3'] == 0

True

In [79]:
%time
instances['attraction_info']

Wall time: 0 ns


,attraction_index,attraction_reward,serve_time
0,e,0,0
1,0,10,32
2,1,7,46
3,2,6,17
4,3,10,22
5,4,7,37
6,h,0,0


In [62]:
instances['visitor_info']

,visitor_index,time_limit
0,0,219
1,1,190
2,2,182
3,3,194
4,4,231
5,5,212
6,6,206
7,7,204
8,8,225
9,9,218


In [104]:
max(instances['visitor_info']['time_limit'])

231

In [61]:
instances['attraction_info']['serve_time'][instances['attraction_info']['attraction_index'] == 4].item()

37

In [86]:
instances['attraction_info']['attraction_reward']


0     0
1    10
2     7
3     6
4    10
5     7
6     0
Name: attraction_reward, dtype: object

In [44]:
instances['attraction_distance_map'].loc[['e', 0, 1],['e', 0, 1]]

,e,0,1
e,0,47,45
0,47,0,34
1,45,34,0


In [66]:
vistor1.enumberate_q(instances['attraction_info'], instances['visitor_info'])['q_index'][0] == (0, 1, 0, 0)

True

In [87]:
instances['attraction_info']

,attraction_index,attraction_reward,serve_time
0,e,0,0
1,0,10,32
2,1,7,46
3,2,6,17
4,3,10,22
5,4,7,37
6,h,0,0


In [90]:
instances['attraction_info']['attraction_reward'][instances['attraction_info']['attraction_index']==3].item()

10

In [96]:
np.inf

inf